# Utilitaire pour la dilution simple
Quelques fonctions pour définir un liquide contenu dans un tube et préparer des dilutions.
La concentration du liquide peut être exprimée en concentration (pour la chimie usuelle) ou en CT (pour la PRC). 

In [2]:
import math

In [3]:
def nice(flt):
    return "{:.2f}".format(flt)

# Définition des classes Liquid et Aliquot

In [4]:
class Liquid:
    def __init__(self, name, tag = ''):
        """define a liquid by it name. Possibility to add a tag (example : pure, dil 1, dil 2)"""
        self.name = name
        self.tag = tag
        
    def __repr__(self):
        return f"<Liquid : {self.name}/{self.tag}>"
        
class Aliquot: 
    def __init__(self, liquid:Liquid, volume, concentration=None, ct=None,  unit_type = 'cc',):
        """La solution est en concentration, sauf si ct est indiquée.
        """
        self.liquid=liquid
        self.volume = volume
        self.unit_type = unit_type
        if unit_type == 'cc':
            self.concentration = concentration
        elif unit_type == 'ct': 
            self.ct = ct
    
    def __repr__(self):
        if self.unit_type == "cc":
            return f"<Aliquot, vol : {self.volume} contenant {self.liquid}>"
        if self.unit_type == "ct":
            return f"<Aliquot, vol : {self.volume} contenant {self.liquid}>"

In [5]:
mater = Liquid("COVID_Pos")
diluent = Liquid("Diluent")
tube1 = Aliquot(mater, 210, 154)
tube2 = Aliquot(diluent, 1100, 0)

# Dilution d'un liquide aliquoté dans un tube

In [6]:
def diluer(tube: Aliquot, dilution = 5, volume_final = 100, tag = 'fille',  comment = False):
    """Préparer une solution fille par dilution d'une solution mère."""
    source_v = volume_final / dilution
    diluent_v = volume_final - source_v 
    liquid_fille = Liquid(tube.liquid.name, tag)
        
    if tube.unit_type == 'cc':
        tube_fille = Aliquot(liquid_fille, volume_final, concentration = tube.concentration/dilution)
    elif tube.unit_type == 'ct':
        tube_fille = Aliquot(liquid_fille, volume_final, ct = tube.ct + math.log(dilution,2))
        
    if comment: 
        print(f"Prélever {nice(source_v)} µl de solution mère {tube}, ajouter {nice(diluent_v)} de diluent ")
        print(f"En sortie on aura le tube : {tube_fille}")
       
    return {'vol_mere' : source_v, 'vol_diluent' : diluent_v, 'tube_fille': tube_fille}

diluer(tube1, 5)
diluer(tube1, 5, volume_final = 55)

{'vol_mere': 11.0,
 'vol_diluent': 44.0,
 'tube_fille': <Aliquot, vol : 55 contenant <Liquid : COVID_Pos/fille>>}

# Calcul sur les CT

## Préparation avec N dilutions ?

In [7]:
class Pipette:
    def __init__(self, nom, mini, maxi):
        self.name = nom
        self.min = mini
        self.max = maxi

P1000 = Pipette("Pip 1000", 100, 900)
P200 = Pipette("Pip 200", 10, 200)

In [8]:
def preparer(vf, ct_cc_cible, tube, n_dil = 1, lst_imposed_dil = None,  comment=False):
    """
    Obtenir un volume vf à concentration finale de CT ct_cc_cible à partir d'un Aliquot en tube dont le CT est connu
    lst_imposed_dil est une liste de dilutions pour forcer les première dilutions
    """
    # produit 
    if tube.unit_type == 'cc':
        dilution = tube.concentration / ct_cc_cible 
    elif tube.unit_type == 'ct':
        dilution = pow(2, ct_cc_cible-tube.ct)
    
    dilution_elementaire = dilution ** (1/n_dil)
    
    if comment:
        print(f"préparer une dilution finale au : {nice(dilution)}")
        
    
    if n_dil == 1 : 
        dico = diluer(tube1 , dilution = dilution, volume_final=vf, comment=True)
        return [ dico ]
    
    elif n_dil > 1 :
        list_dil = []
        if lst_imposed_dil:
            # Verifier n_dil sup a liste
            assert len(lst_imposed_dil) < n_dil
            if comment: 
                print("Nous allons préparer les dilution imposées puis les libres")
            sub_tube = None # {} # contiendra la succession des dilutions
            tube_en_cours_de_dil = tube
            for i in range(n_dil):                
                if i < len(lst_imposed_dil):
                    print("dilution imposée", i)
                
                    import pdb
                    pdb.set_trace()
                
                    sub_tube = diluer(tube_en_cours_de_dil, dilution=lst_imposed_dil[i][0], tag="imposé"+str(i+1))
                    
                    
                    list_dil.append(sub_tube)
                    print("CONCENTRATION", sub_tube['tube_fille'].concentration, 
                          sub_tube['tube_fille'].unit_type, 
                         sub_tube['tube_fille'].ct)
                    
                else:
                    list_dil.append("Et il reste une dilution ! ")
                    
        else:
        # N dilutions successives identiques
            if comment: 
                print(f"Nous allons préparer {n_dil} dilutions au {nice(dilution_elementaire)}")
            for i in range(0, n_dil):
                print("dilution", i+1)
                dico = diluer(tube, dilution=dilution_elementaire, tag='dil_' + str(i+1), volume_final=vf, comment=True)  
                list_dil.append(dico)
                
        return list_dil
    
tube_ct_mere = Aliquot(mater, 500, ct=20.5, unit_type='ct')
preparer(200, 35, tube_ct_mere, comment=True)
print()
preparer(800, 35, tube_ct_mere, n_dil=3, comment=True)
print()
preparer(800, 35, tube_ct_mere, n_dil=3, lst_imposed_dil = [[100, 500], [50,100]], comment=True)

préparer une dilution finale au : 23170.48
Prélever 0.01 µl de solution mère <Aliquot, vol : 210 contenant <Liquid : COVID_Pos/>>, ajouter 199.99 de diluent 
En sortie on aura le tube : <Aliquot, vol : 200 contenant <Liquid : COVID_Pos/fille>>

préparer une dilution finale au : 23170.48
Nous allons préparer 3 dilutions au 28.51
dilution 1
Prélever 28.06 µl de solution mère <Aliquot, vol : 500 contenant <Liquid : COVID_Pos/>>, ajouter 771.94 de diluent 
En sortie on aura le tube : <Aliquot, vol : 800 contenant <Liquid : COVID_Pos/dil_1>>
dilution 2
Prélever 28.06 µl de solution mère <Aliquot, vol : 500 contenant <Liquid : COVID_Pos/>>, ajouter 771.94 de diluent 
En sortie on aura le tube : <Aliquot, vol : 800 contenant <Liquid : COVID_Pos/dil_2>>
dilution 3
Prélever 28.06 µl de solution mère <Aliquot, vol : 500 contenant <Liquid : COVID_Pos/>>, ajouter 771.94 de diluent 
En sortie on aura le tube : <Aliquot, vol : 800 contenant <Liquid : COVID_Pos/dil_3>>

préparer une dilution finale a

ipdb> n
> /tmp/ipykernel_35459/3353032244.py(12)diluer()
     10         tube_fille = Aliquot(liquid_fille, volume_final, ct = tube.ct + math.log(dilution,2))
     11 
---> 12     if comment:
     13         print(f"Prélever {nice(source_v)} µl de solution mère {tube}, ajouter {nice(diluent_v)} de diluent ")
     14         print(f"En sortie on aura le tube : {tube_fille}")

ipdb> tube_fille
<Aliquot, vol : 100 contenant <Liquid : COVID_Pos/imposé1>>
ipdb> tube_fille.unit_type
'cc'
ipdb> c


AttributeError: 'Aliquot' object has no attribute 'ct'

# Tests

In [1]:
mater = Liquid("COVID")
diluent = Liquid("Diluent")
tube1 = Aliquot(mater, 200, 154)
tube2 = Aliquot(diluent, 1000, 0)

print(tube1)
print(tube2)
preparer(200, 15, tube1)

preparer(200, 35, tube_ct_mere, comment = True)

NameError: name 'Liquid' is not defined